In [1]:
from trnasimtools.serialize import SerializeTwoCodonSingleTranscript
import os

In [2]:
date="june-5-2022"

In [3]:
#!mkdir ../yaml/june-5-2022
#!mkdir ../output/june-5-2022

In [4]:
# simulation parameters
time_limit = 500
time_step = 5
transcript_len = 100
transcript_copy_number = 100
ribosome_copy_number = 100
total_trna = 100
ribosome_binding_rate = 1
trna_charging_rates = [0.0001, 0.00025, 0.0005, 0.00075, 0.001, 0.0025, 0.005, 0.0075, 0.01, 0.025, 0.05, 0.075, 0.1, 0.25, 0.5, 0.75, 1.0, 2.5, 5.0, 7.5, 10.0]

In [8]:
# Write simulation configuration file - I'm using the TwoCodon class I wrote, but the transcript is just going to consist of a single codon repeated 100 times
trna1_prop, trna2_prop = (1.0, 0.0)
codon1_prop, codon2_prop = (1.0, 0.0)
serializer = SerializeTwoCodonSingleTranscript(transcript_len=transcript_len,
                                           codon_comp=(codon1_prop, codon2_prop),
                                           trna_proportion=(trna1_prop, trna2_prop),
                                           time_limit=time_limit,
                                           time_step=time_step)
serializer.serialize(f"../yaml/{date}")
config = f"../yaml/{date}/{serializer.filename()}"

In [11]:
#!mkdir ../output/june-5-2022/cell_volume_8e-16

In [12]:
cell_volume = "8e-16"
with open(f"../scripts/cmd_files/{date}_{cell_volume}.txt", "w") as stream:
        for trna_charging_rate in trna_charging_rates:
            for seed in range(1, 6):
                cmd = f"python3 twocodonsingletranscript_cellvol.py {config} {seed} {transcript_copy_number} " + \
                      f"{ribosome_copy_number} {total_trna} {ribosome_binding_rate} {trna_charging_rate} {trna_charging_rate} " + \
                      f"{cell_volume} ../output/{date}/cell_volume_{cell_volume}"
                stream.write(cmd)
                stream.write("\n")